In [33]:
from __future__ import print_function
import tensorflow as tf
import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import import_ipynb
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose,Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.activations import softmax
from data import load_train_data, load_test_data

K.set_image_data_format('channels_last')  # TF dimension ordering in this code

img_rows = 96
img_cols = 96

smooth = 1.


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    print(tf.shape(conv4))
    #split
    td=tf.reshape(conv4,(256,144))
    tx=softmax(td,axis=1)
    q=1.5
    
    
    n=tx.shape[0]
    temp=[]
    
    for i in range(n):
        temp.append(tf_tsallis_entropy(tx[i],td[i],q))
    
    
    before_gap=tf.Variable(temp)
    
    dense_nodes=tf.reduce_sum(before_gap,axis=1)/144
    
    dense_nodes=tf.reshape(dense_nodes,shape=(1,256))
    
    dense1=Dense(48,input_shape=(1,),activation='relu')(dense_nodes)
    dense2=Dense(2,activation='softmax')(dense1)
    
    
    
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10,dense2])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model


def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p


def train_and_predict():
    imgs_train,imgs_mask_train,imgs_value_train=load_train_data()

    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)

    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    imgs_train -= mean
    imgs_train /= std

    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]

    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(imgs_train, y=[imgs_mask_train,imgs_value_train], batch_size=32, epochs=1, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[model_checkpoint])

    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    imgs_test, imgs_id_test = load_test_data()
    imgs_test = preprocess(imgs_test)

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('weights.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    imgs_mask_test = model.predict(imgs_test, verbose=1)
    np.save('imgs_mask_test.npy', imgs_mask_test)

    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)
    pred_dir = 'preds'
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
    for image, image_id in zip(imgs_mask_test, imgs_id_test):
        image = (image[:, :, 0] * 255.).astype(np.uint8)
        imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)

if __name__ == '__main__':
    train_and_predict()

------------------------------
Creating and compiling model...
------------------------------
Tensor("Shape_8:0", shape=(4,), dtype=int32)
p^q/(q-1)---> Tensor("RealDiv_6:0", shape=(144,), dtype=float32)
-p^q/(q-1)----> Tensor("Neg_8:0", shape=(144,), dtype=float32)


ValueError: in user code:

    <ipython-input-31-d21004161531>:24 getsegment  *
        if new_p[i] == new_p[j]:
    /home/gr8_saurav/anaconda3/lib/python3.7/site-packages/tensorflow/python/autograph/operators/control_flow.py:951 if_stmt
        _tf_if_stmt(cond, body, orelse, get_state, set_state, symbol_names, nouts)
    /home/gr8_saurav/anaconda3/lib/python3.7/site-packages/tensorflow/python/autograph/operators/control_flow.py:999 _tf_if_stmt
        set_state(final_cond_vars)
    /tmp/tmplebfbr37.py:54 set_state_1
        (temp_p[j], flag_p[j]) = vars_

    ValueError: setting an array element with a sequence.


In [31]:
@tf.function
def getsegment(new_p):
    n = new_p.shape[0]

#     for i in range(n-1):
#         if new_p[i] != new_p[i+1]:
#             count=count+1

    class1 = 0
    temp_p = np.zeros(n, dtype=np.uint32)
    flag_p = np.zeros(n,dtype=np.uint32)

#     for i in range(n):
#         temp = np.array(0)
#         flag_p.append(temp)
  #0.14,0.23,0.14,0.35,0.23,0.14
    for i in range(n):

        if flag_p[i] != 1:
            temp_p[i] = class1
            flag_p[i] = 1

        for j in range(i+1,n):
            if new_p[i] == new_p[j]:
                temp_p[j] = class1
                flag_p[j] = 1

        class1 = class1+1




    final_p = tf.Variable(initial_value=temp_p)
    print("tensor is",final_p)
    return final_p

def tf_calculate_entropy(segment_map,segment_ids):
  

    n = segment_map.shape[0]
    m = segment_ids.shape[0]


    entropy = []
    for j in range(m):
        temp=np.array(0)
        entropy.append(temp)


    for i in range(n):

        for j in range(m):

            if segment_ids[j] == i:
                entropy[j]=segment_map[i]

    
    final_entropy = tf.Variable(entropy)

    print("final entropy",final_entropy)
        
    return final_entropy

In [32]:
def tf_log_q(x,q):
    safe_x = tf.maximum(x,1e-6)

    log_q_x = tf.cond(tf.equal(q,1.),true_fn=lambda: tf.log(safe_x),false_fn=lambda: (tf.pow(safe_x,q))/(q-1))
    return log_q_x


def tf_tsallis_entropy(p,input_row,q):
    new_p = tf_log_q(p,q) #P^q/(q-1)
    print("p^q/(q-1)--->",new_p)
    new_p = -new_p #-p^q/(q-1)
    print("-p^q/(q-1)---->",new_p)
    segment_ids = getsegment(new_p) #got the segment ids
    print("ids is",segment_ids)
    segment_map = tf.math.segment_sum(new_p,segment_ids) #got the segment sum as per class....sum(-p^q/(q-1)) 
    segment_map = 1+segment_map  # final sum ...1+sum(-p^q/(q-1))
    print("new segement map is",segment_map)   
    final_tsallis_entropy = tf_calculate_entropy(segment_map,segment_ids) #repeat the sums
    lamda = 1-final_tsallis_entropy/(tf.math.reduce_max(final_tsallis_entropy))
    print("wieghts and ps",lamda,input_row)
    improved_p = tf.multiply(input_row,lamda)
    print("final wieghted p",improved_p)
    return improved_p

In [20]:
t=np.zeros(12,dtype=np.uint64)

In [34]:
t[2]=1

In [35]:
t

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint64)